In [1]:
import requests
import defs
import pandas as pd

In [2]:
# Request has a session object, help maintain persisteng connection
session = requests.Session()

In [3]:
ins_df = pd.read_pickle("instrument.pkl")

In [4]:
our_curr = ['EUR','USD','GBP','JPY','CHF','NZD', 'CAD']

In [6]:
# Get all possible tradabl;e pairs
ins_df.shape
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         123 non-null    object
 1   type         123 non-null    object
 2   displayName  123 non-null    object
 3   pipLocation  123 non-null    int64 
 4   marginRate   123 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.9+ KB


In [10]:
ins_df.name.unique()

array(['USD_SGD', 'EUR_SEK', 'HKD_JPY', 'AUD_USD', 'USD_CAD',
       'US2000_USD', 'USB10Y_USD', 'NZD_USD', 'XAG_EUR', 'XAG_GBP',
       'HK33_HKD', 'NZD_SGD', 'USD_NOK', 'USD_CNH', 'USB02Y_USD',
       'XCU_USD', 'SGD_CHF', 'SPX500_USD', 'GBP_JPY', 'USD_TRY',
       'AUD_JPY', 'CN50_USD', 'ZAR_JPY', 'XAG_JPY', 'SGD_JPY', 'GBP_ZAR',
       'USD_JPY', 'EUR_TRY', 'EUR_JPY', 'AUD_SGD', 'XAG_NZD', 'WTICO_USD',
       'XAG_AUD', 'EUR_NZD', 'GBP_HKD', 'CHF_JPY', 'EUR_HKD', 'GBP_CAD',
       'XAU_HKD', 'XAU_JPY', 'USD_THB', 'GBP_CHF', 'AUD_CHF', 'ESPIX_EUR',
       'NZD_CHF', 'AUD_HKD', 'XAG_HKD', 'USD_CHF', 'XAG_CHF', 'CAD_HKD',
       'CH20_CHF', 'XAU_SGD', 'CORN_USD', 'EUR_CHF', 'EUR_SGD', 'NZD_CAD',
       'CHINAH_HKD', 'GBP_AUD', 'USD_PLN', 'EUR_ZAR', 'NAS100_USD',
       'TRY_JPY', 'SOYBN_USD', 'XAU_XAG', 'AU200_AUD', 'XAU_CAD',
       'EU50_EUR', 'EUR_AUD', 'NL25_EUR', 'USD_ZAR', 'CAD_JPY', 'NZD_HKD',
       'XAU_NZD', 'USD_CZK', 'USD_DKK', 'XAU_GBP', 'USD_SEK', 'GBP_SGD',
       'EUR_

In [13]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers= defs.SECURE_HEADER)
    return response.status_code, response.json()



In [18]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    our_data= []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        #print(new_dict)
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"]= candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)


In [27]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pkl")


In [31]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 5000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)


In [32]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

EUR_USD loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
EUR_GBP loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
EUR_JPY loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
EUR_CHF loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
EUR_NZD loaded 5000 candles from 2023-01-17T13:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
EUR_CAD loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
USD_JPY loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
USD_CHF loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
USD_CAD loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
GBP_USD loaded 5000 candles from 2023-01-17T14:00:00.000000000Z to 2023-11-03T20:00:00.000000000Z
GBP_JPY loaded 5000 